# Processing of TopAGNPS data for CCHE1D

## Summary

### Inputs
The `topagnps2cche1d` module is a set of Python functions that process some of the outputs of TopAGNPS i.e.:
- `AnnAGNPS_Reach_IDs.asc`
    - The raster file of the different reaches obtained from processing a DEM file
- `AgFlow_Reach_Data.csv`
    - csv file containing information about each reach, the position of the upstream and downstream row/column coordinates of each reach, the receiving reach etc.
- `NETW.asc`
    - Raster file indicating what is the Strahler number of each pixel of each reach

### Outputs
The module produces the:
- `*_nodes.dat`
- `*_channel.dat`
- `*_link.dat`
- `*_reach.dat`
- `*_csec.dat`
- `*_csprf.dat`

files that can be used as inputs by CCHE1D

### Assumptions made
- This tool assumes that every junction consists strictly of two upstream inflows. This is insured by TopAGNPS.
- For now, every node is assigned a default cross-section

### Additional tools
Functions to read, process and visualize `.asc` raster files as well as CCHE1D `*_nodes.dat` files are provided in the module. Functions to convert row/col to latitude/longitude coordinates and inversely are provided making use of the `osgeo.gdal` and `affine` modules.


### Loading the module

In [1]:
import topagnps2cche1d.tools as t2c
import plotly.express as px

### Files location

In [2]:
# Main Folder
srcfolder = '../input_data/topagnps_ohio_files/'

# File names
filepath_annagnps_reach_ids = srcfolder + 'AnnAGNPS_Reach_IDs.asc'
filepath_agflow = srcfolder + 'AgFlow_Reach_Data.csv'
# filepath_flovec = srcfolder + 'FLOVEC.asc'
filepath_netw = srcfolder + 'NETW.asc'
filepath_dednm = srcfolder + 'DEDNM.asc'

### Reading the reach data
- `img` is a Numpy array that represents the raster of the `AnnAGNPS_Reach_IDs.asc` file
- `geoMatrix` is a 6-tuple holding the coefficients of the affine transformation to convert row and columns to latitude and longitude
- `ncols` and `nrows` are the dimensions of the raster
- `nodataval` is the value used in the raster to represent the absence of data
- `dataset` is a `gdal` object that holds all the data related to the raster 

In [3]:
img, geoMatrix, ncols, nrows, nodataval, dataset = t2c.read_esri_asc_file(filepath_annagnps_reach_ids)

### Testing of the row/col conversion tool

In [4]:
# Coordinates of a known point (the outlet) and the computed coordinates
rowinput, colinput = 83, 464
truelat, truelon = 4476008.33, 717164.897

latcalc, loncalc = t2c.rowcol2latlon_esri_asc(geoMatrix, rowinput, colinput, oneindexed=True)
print(f"(row, col) = ({rowinput}, {colinput}) maps to ({latcalc}, {loncalc}); Error: calc-true = ({latcalc-truelat}, {loncalc-truelon})")

(row, col) = (83, 464) maps to (4476008.33, 717164.897); Error: calc-true = (0.0, 0.0)


### Visualization of the reach network as numbered by TopAGNPS

In [5]:
fig = t2c.visualize_reaches_id(img, geoMatrix, title='Initial Reach IDs', renderer='browser')

### Tree traversal strategies
<img src='content/Sorted_binary_tree_ALL_RGB.svg.png' alt=tree-traversal width='400'/>

Depth-first traversal (dotted path) of a binary tree: 
- Pre-order (node visited at position red 🔴):     
    - F, B, A, D, C, E, G, I, H; 
- In-order (node visited at position green 🟢):
    - A, B, C, D, E, F, G, H, I; 
- Post-order (node visited at position blue 🔵):
    - A, C, E, D, B, H, I, G, F.

Assuming each node is a reach and the sub-nodes are the inflows (ordered from left to right in the counter-clockwise order when *looking* upstream of a junction)

#### Facts:
- TopAGNPS numbers the reaches from the outlet in an **pre-order** fashion but traveling in the opposite direction of the dotted line (🔵).
- CCHE1D requires a numbering of reaches (channels) in a **post-order** fashion (🔵, following the dotted line)

### Building of the channel network

The channel network is constructed for CCHE1D using the connectivity data in the `Agflow_Reach_Data.csv` file

In [6]:
dfagflow = t2c.read_agflow_reach_data(filepath_agflow)
dfagflow.head(n=10)

,Reach_ID,Upstream_End_Row,Upstream_End_Column,Downstream_End_Row,Downstream_End_Column,Receiving_Reach,Drainage_Area_[ha],Average_Elevation_[m],Reach_Length_[m],Distance_Upstream_End_to_Outlet_[m],Distance_Downstream_End_to_Outlet_[m],Reach_Slope_[m/m],Contributing_Cell_ID_Source,Contributing_Cell_ID_Left,Contributing_Cell_ID_Right
0,1,83,464,83,464,1,145.80,294.30,0.00,0.00,0.00,0.00001,0,0,0
1,2,84,465,83,464,1,145.80,294.30,7.24,7.24,0.00,0.00001,0,1,0
2,3,155,515,85,466,2,86.10,294.46,311.74,318.98,7.24,0.00097,0,3,2
3,4,134,617,156,516,3,22.72,295.30,359.43,678.41,318.98,0.00251,0,5,4
4,5,95,654,133,618,4,4.48,295.50,168.25,846.66,678.41,0.00001,6,8,7
5,6,130,668,134,618,4,7.12,295.50,174.64,853.05,678.41,0.00001,9,11,10
6,7,215,510,156,515,3,59.03,294.81,196.15,515.13,318.98,0.00357,0,13,12
7,8,251,550,216,511,7,37.41,295.30,170.01,685.14,515.13,0.00001,0,15,14
8,9,274,575,251,551,8,26.92,295.30,103.58,788.72,685.14,0.00001,0,17,16
9,10,248,600,274,576,9,3.23,295.33,116.10,904.81,788.72,0.00171,18,20,19


### Visualization of Strahler Number for the network

The following diagram illustrates the Strahler number for a river network and is an indication of the network complexity

<img src='content/strahler.png' align="center" alt=tree-traversal width='500'/>



In [7]:
img_strahler, geoMatrix, _, _, _, _, = t2c.read_esri_asc_file(filepath_netw)
fig = t2c.visualize_strahler_number(img_strahler, geoMatrix, renderer='browser')

### Generating the CCHE1D files

All the heavy lifting is done inside the `convert_topagnps_output_to_cche1d_input` and `create_cche1d_tables` functions.

The outputs are the Pandas DataFrame of the nodes, channel, and link files. `img_reach` is a Numpy array with the reordered reach numbering

#### Generating a default cross-section for all nodes of the network

CP_W and CP_Z are the x and y coordinates of the cross-section and CP_RGH the Manning's roughness coefficient

In [8]:
default_xsection = {'type' : 'default',
                    'CP_Ws': [-43, -35, -13, -10, 10, 13, 35, 43],
                    'CP_Zs': [6, 2, 2, 0, 0, 2, 2, 6],
                    'CP_RGHs': [0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025]}

fig = px.line(x = default_xsection['CP_Ws'], y = default_xsection['CP_Zs'], markers='o', title = 'Default Cross-Section')
fig.update_layout(xaxis_title='x (m)', yaxis_title='y (m)')
fig.show()

In [9]:
distance = 50
min_strahler = 1
# df_nodes, df_channel, df_link, df_reach, df_csec, df_csprf, img_reach = t2c.convert_topagnps_output_to_cche1d_input(filepath_agflow, filepath_flovec, filepath_annagnps_reach_ids, filepath_netw, default_xsection, min_strahler, distance)
df_nodes, df_channel, df_link, df_reach, df_csec, df_csprf, img_reach, cche1d_to_annagnps_reaches = t2c.convert_topagnps_output_to_cche1d_input(filepath_agflow, filepath_annagnps_reach_ids, filepath_netw, default_xsection, min_strahler, distance)

#### Reordered reaches
Note that the reach containing the outlet is the reach with the highest reach number

In [20]:
reaches_cche1d_topagnps = t2c.dict_cche1d_annagnps_to_df(cche1d_to_annagnps_reaches)
reaches_cche1d_topagnps

,CCHE1D_Reach,TopAGNPS_Reach
0,27,2
1,9,20
2,26,3
3,24,5
4,23,6
5,25,4
6,22,7
7,1,28
8,2,27
9,7,22


In [11]:
fig = t2c.visualize_reaches_id(img_reach, geoMatrix, title='New Reach IDs', renderer='browser')

#### Visualization of the nodes as seen by CCHE1D:

| ND_TYPE | Meaning |
|---------|---------|
| 0       | Source node |
| 2       | *Upstream* junction node |
| 3       | *Downstream/Receiving* junction node |
| 6       | Normal node |
| 9       | Outlet node |

Note: The coordinates of points 2 and 3 are the exact same for any given junction


In [12]:
fig = t2c.visualize_cche1d_nodes(df_nodes, renderer='browser')

### Writing of the files

In [15]:
outputfolder = '../input_data/topagnps_ohio_files/top2cche1d_outputs/' # Location to write the files
casename = 'ohio' # Name of the case for these files

In [16]:
t2c.write_cche1d_dat_file(f'{outputfolder}{casename}_min_strahler_{min_strahler}', df_nodes)
t2c.write_cche1d_dat_file(f'{outputfolder}{casename}_min_strahler_{min_strahler}', df_channel)
t2c.write_cche1d_dat_file(f'{outputfolder}{casename}_min_strahler_{min_strahler}', df_link)
t2c.write_cche1d_dat_file(f'{outputfolder}{casename}_min_strahler_{min_strahler}', df_reach)
t2c.write_cche1d_dat_file(f'{outputfolder}{casename}_min_strahler_{min_strahler}', df_csec)
t2c.write_cche1d_dat_file(f'{outputfolder}{casename}_min_strahler_{min_strahler}', df_csprf)

../topagnps_ohio_files/top2cche1d_outputs/ohio_min_strahler_1_nodes.dat successfully written
../topagnps_ohio_files/top2cche1d_outputs/ohio_min_strahler_1_channel.dat successfully written
../topagnps_ohio_files/top2cche1d_outputs/ohio_min_strahler_1_link.dat successfully written
../topagnps_ohio_files/top2cche1d_outputs/ohio_min_strahler_1_reach.dat successfully written
../topagnps_ohio_files/top2cche1d_outputs/ohio_min_strahler_1_csec.dat successfully written
../topagnps_ohio_files/top2cche1d_outputs/ohio_min_strahler_1_csprf.dat successfully written


### Channel network when imported in CCHE1D GUI

<img src='content/cche1d_gui.png' align="center" alt=tree-traversal width='1200'/>